In [ ]:
import requests
from bs4 import BeautifulSoup
from nltk.probability import FreqDist
import nltk

from datetime import datetime
import pytz

!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3
!pip3 install konlpy

In [2]:
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
from collections import Counter
import re


okt = Okt()
# 한국 시간대로 설정하기
korea_tz = pytz.timezone('Asia/Seoul')

# 현재 시간을 가져오기
current_datetime = datetime.now(korea_tz)

# 한국 시간으로 현재 날짜와 시간을 출력하기
print("한국 시간 기준 현재 날짜와 시간:", current_datetime)

# 첫 번째 URL에서 기사 제목 가져오기
def get_article_title(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.select_one('#content > div > div.content > div > div.news_headline > h4')
    return title.get_text().strip() if title else None

# 두 번째 URL에서 기사 제목들 가져오기
def get_article_titles(url):
    titles = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.select('a.title > span')
    for article in articles:
        title_text = article.get_text(strip=True)
        titles.append(title_text)
    return titles

# 기사 내용 불러오기
def get_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    article_text = ''
    article = soup.find('div', {'class': 'news_end'})
    if article:
        for unwanted_1 in article.find_all('em', {'class': 'img_desc'}):
            unwanted_1.extract()

        for unwanted_2 in article.find_all(string=re.compile(r'\[.*?\]')):
            unwanted_2.extract()

        unwanted_3 = article.find_all(['div', 'p'], {'class': ['source', 'byline', 'reporter_area', 'copyright', 'categorize', 'promotion']})
        for unwanted in unwanted_3:
            unwanted.extract()

        article_text = article.get_text(separator='\n', strip=True)

    return article_text

# TF-IDF 벡터화 및 유사도 계산하기
def calculate_tfidf_similarity(title1, titles2):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([title1] + titles2)
    similarity_matrix = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    return similarity_matrix

# 유사한 제목들을 출력하기
def find_similar_titles(similarity_matrix, title1, titles2, threshold=0.03):
    similar_titles = []
    for i, similarity_score in enumerate(similarity_matrix.flatten()):
        if similarity_score > threshold:
            similar_titles.append((title1, titles2[i], similarity_score))
    return similar_titles

# 두 개의 URL에서 기사 제목을 가져오기
url1 = input("원하는 네이버스포츠 기사를 입력하세요: ")
url2 = "https://sports.news.naver.com/kbaseball/news/index?isphoto=N"
title1 = get_article_title(url1)
titles2 = get_article_titles(url2)
for t2 in titles2:
  if title1 == t2:
    titles2.remove(t2)
print(f"기사 제목: {title1}")

if title1 and titles2:
    # TF-IDF 유사도 계산하기
    similarity_matrix = calculate_tfidf_similarity(title1, titles2)

    # 유사한 제목들을 출력하기
    similar_titles = find_similar_titles(similarity_matrix, title1, titles2)
    if similar_titles:
        print("유사한 제목들:")
        for title_pair in set(similar_titles):
            print(f"URL 기사 제목: {title_pair[0]}")
            print(f"기사 2 제목: {title_pair[1]} (유사도: {title_pair[2]:.4f})")
            print("----------")
    else:
        print("유사한 제목이 없습니다.")
else:
    print("기사 제목을 가져올 수 없습니다.")

# 기사 요약하기
def summarize_article(text):
    tokens = okt.nouns(text)
    word_freq = Counter(tokens)

    most_freq_words = [word[0] for word in word_freq.most_common(3)]

    sentences = text.split('.')

    summary_sentences = []
    for sentence in sentences:
        for word in most_freq_words:
            if word in sentence and sentence.strip() not in summary_sentences and len(summary_sentences) < 3:
                summary_sentences.append(sentence.strip())

    summary = '. '.join(summary_sentences)
    return summary

# 기사 내용을 가져오기
article_text = get_article_text(url1)

# 기사 내용 요약
summary = summarize_article(article_text)

print("기사 요약:\n", summary)



한국 시간 기준 현재 날짜와 시간: 2023-12-17 10:43:16.485791+09:00
원하는 네이버스포츠 기사를 입력하세요: https://sports.news.naver.com/news?oid=003&aid=0012270766
기사 제목: SSG, 에레디아·엘리아스와 재계약…2024시즌 외인 구성 완료
유사한 제목들:
URL 기사 제목: SSG, 에레디아·엘리아스와 재계약…2024시즌 외인 구성 완료
기사 2 제목: "재계약 보험? 데려올 선수가 없어요" 구단들 역대급 구인난, 어디서부터 꼬였을까? (유사도: 0.0766)
----------
URL 기사 제목: SSG, 에레디아·엘리아스와 재계약…2024시즌 외인 구성 완료
기사 2 제목: 난세에 중책…김재현 SSG 단장 "진정성 갖고 팬들에게 다가가겠다" (유사도: 0.0972)
----------
URL 기사 제목: SSG, 에레디아·엘리아스와 재계약…2024시즌 외인 구성 완료
기사 2 제목: 돌아온 MVP+재계약 성공한 좌승사자, LG 왕조 건립 막아서나 (유사도: 0.0826)
----------
기사 요약:
 지난달 새 외국인 투수 더거 영입
SSG는 외국인 타자 에레디아와 총액 150만달러(계약금 15만달러·연봉 115만달러·옵션 20만달러)에, 외국인 투수 엘리아스와 총액 100만달러(계약금 10만달러·연봉 65만달러·옵션 25만달러)에 각각 재계약을 체결했다고 17일 밝혔다. 지난달 말 새 외국인 투수 로버트 더거와 총액 90만 달러(계약금 10만달러·연봉 65만달러·옵션 15만달러)에 계약한 SSG는 이로써 2024시즌 외국인 선수 구성을 모두 마쳤다. 지난해 12월 신규 외국인 선수 상한선인 100만달러를 꽉 채워 계약했던 엘리아스는 50만달러 오른 금액에 사인했다
